# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import pandas as pd
from azureml.core import Dataset
import sys
from azureml.automl.runtime.onnx_convert import OnnxConverter

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment
from azureml.core.model import Model
from azureml.automl.core.shared import constants


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure
This code sets up an machine learning AutoML experiment to develop a model to make such predictions. It also, deploys the model to Azure and 
tests the endpoint webservice. The data was obtained via the Kaggle website and is external to Udacity

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project-automl'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')



2023-04-09:13:27:48,567 INFO     [workspace.py:291] Found the config file in: /config.json
2023-04-09:13:27:49,35 INFO     [clientbase.py:192] Created a worker pool for first use


In [3]:
cluster_name = "udacityMLCluster"

# Create compute cluster

#verify that cluster does not exist
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', cluster_name)
    compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
    compute_max_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 4)

    vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'Standard_D2_V2')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes= compute_min_nodes, max_nodes=compute_max_nodes)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
#Obtaining data file from github repository
webpath = "https://raw.githubusercontent.com/Ed-Ramos/ML-with-Azure-Capstone/master/heart_failure_clinical_records_dataset.csv"
df = pd.read_csv(webpath)
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1


In [5]:
sys.path.append('train')
from train import clean_data

# Use the clean_data function in the train.py script to clean the data.
x, y = clean_data(df)

In [6]:
#joining x and y datframes into one dataframe
data = x.join(y)

#create a folder to save data file
if not os.path.isdir('data'):
    os.mkdir('data')
    
# Save the training data to a csv to be uploaded to the datastore
pd.DataFrame(data).to_csv("data/training_data.csv", index=False)

# setting the default datastore
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='heart_disease', overwrite=True, show_progress=True)
 
# Upload the training data as a tabular dataset for access during training on remote compute
training_data = Dataset.Tabular.from_delimited_files(path=ds.path('heart_disease/training_data.csv'))
label = "DEATH_EVENT"

2023-04-09:13:28:01,536 INFO     [datastore_client.py:991] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f7560728700>
2023-04-09:13:28:01,913 WARNING  [_dataset_deprecation.py:89] "Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.
2023-04-09:13:28:01,915 INFO     [azure_storage_datastore.py:923] Called AzureBlobDatastore.upload
2023-04-09:13:28:01,931 INFO     [azure_storage_datastore.py:372] Uploading an estimated of 3 files
2023-04-09:13:28:02,183 INFO     [azure_storage_datastore.py:372] Uploading ./data/.amlignore
2023-04-09:13:28:02,184 INFO     [azure_storage_datastore.py:372] Uploaded ./data/.amlignore, 1 files out of an estimated total of 3
2023-04-09:13:28:02,185 INFO     [azure_storage_datastore.py:372] Uploading ./data/.amlignore.amlt

## AutoML Configuration

TODO: Explain why you chose the automl settings and configuration you used below.

The autoML setting were chosen as follows:
experiment_timeout_minutes: to save on compute resources, using 20 minutes as this is sufficient for the purpose of this project
- primary_metric: Since this is a classification problem, using the accuracy metric which matches the metric used in hyperdrive experiment
- n_cross_validations: Using cross validation as a resampling method to use different portion of data to test and train the model on different iterations. Using n=5 is sufficient for this data size
- iteration_timeout_minutes: This setting guards against iterations that have issues and are taking too long to complete and thus saves compute resources
- max_concurrent_iterations:  This setting provides compute efficiency. 

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":20,
    "primary_metric":'accuracy',
    "n_cross_validations":5,
    "iteration_timeout_minutes":5,
    "max_concurrent_iterations":4,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task='classification',
                compute_target= compute_target,
                training_data=training_data,
                label_column_name=label,
                enable_onnx_compatible_models=True,
                **automl_settings
                )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on udacityMLCluster with default configuration
Running on remote compute: udacityMLCluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project-automl,AutoML_fbe6c560-b146-4330-bf2e-0ac8692d1dda,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTI

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_automl_run, best_automl_model = remote_run.get_output(return_onnx_model=True)
print(best_automl_run)
print(best_automl_model)

#saving model in onnx format in outputs directory
OnnxConverter.save_onnx_model(best_automl_model, file_path="./outputs/automl_model.onnx")

Run(Experiment: capstone-project-automl,
Id: AutoML_fbe6c560-b146-4330-bf2e-0ac8692d1dda_43,
Type: azureml.scriptrun,
Status: Completed)
ir_version: 8
producer_name: "AutoML"
producer_version: "1.48.0"
domain: "ai.onnx"
model_version: 0
doc_string: "{\"AutoMLSDKVer\": \"1.48.0\", \"ExperimentName\": \"capstone-project-automl\", \"RunId\": \"AutoML_fbe6c560-b146-4330-bf2e-0ac8692d1dda_43\", \"PipeId\": \"__AutoML_Ensemble__\"}"
graph {
  node {
    input: "age"
    output: "variable_c0_t0"
    name: "Imputer"
    op_type: "Imputer"
    attribute {
      name: "imputed_value_floats"
      floats: 60.833892822265625
      type: FLOATS
    }
    attribute {
      name: "replaced_value_float"
      f: nan
      type: FLOAT
    }
    domain: "ai.onnx.ml"
  }
  node {
    input: "creatinine_phosphokinase"
    output: "creatinine_phosphokinase_scaled"
    name: "Scaler"
    op_type: "Scaler"
    attribute {
      name: "offset"
      floats: 0.0
      type: FLOATS
    }
    attribute {
      n

In [11]:
#model properties
best_run_properties = best_automl_run.get_properties()
print(best_run_properties)

{'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-project-automl\',\'compute_target\':\'udacityMLCluster\',\'subscription_id\':\'dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f\',\'region\':\'eastus\',\'spark_service\':None}","ensemble_run_id":"AutoML_fbe6c560-b146-4330-bf2e-0ac8692d1dda_43","experiment_name":"capstone-project-automl","workspace_name":"udacitymlworkspace","subscription_id":"dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f","resource_group_name":"udacityml"}}]}', 'training_percent': '100', 'predicted_cost': None, 'iteration': '43', '_aml_system_scenario_identification': 'Remote.Child', '_azureml.ComputeTargetType': 'aml

In [12]:
#TODO: Register and save the best model

best_automl_run.download_file('./outputs/model.pkl', './outputs/capstone_automl_model.pkl')

model = best_automl_run.register_model(model_name='best_AutoML_final', model_path='./outputs/model.pkl')

print(model)

#run.complete()

Model(workspace=Workspace.create(name='udacitymlworkspace', subscription_id='dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f', resource_group='udacityml'), name=best_AutoML_final, id=best_AutoML_final:1, version=1, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
#Obtaining CODA evirionment data used in run and saving it to directory
best_automl_run.download_file(constants.CONDA_ENV_FILE_PATH, 'heart_dependencies.yml')

environment = Environment.from_conda_specification('heart_dependencies', 'heart_dependencies.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=environment)

service_name = 'capstone-automl-deploy'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

service = Model.deploy(workspace=ws,
                          name=service_name,
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=aci_config,
                          overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-04-09 13:49:27+00:00 Creating Container Registry if not exists.
2023-04-09 13:49:27+00:00 Registering the environment.
2023-04-09 13:49:29+00:00 Use the existing image.
2023-04-09 13:49:29+00:00 Generating deployment configuration.
2023-04-09 13:49:30+00:00 Submitting deployment to compute.
2023-04-09 13:49:37+00:00 Checking the status of deployment capstone-automl-deploy..
2023-04-09 13:51:42+00:00 Checking the status of inference endpoint capstone-automl-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
print(service.state)

Healthy


In [15]:
service.get_logs()

'/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n2023-04-09T13:51:27,572706800+00:00 - rsyslog/run \n2023-04-09T13:51:27,572706800+00:00 - iot-server/run \n2023-04-09T13:51:27,579559700+00:00 - gunicorn/run \n2023-04-09T13:51:27,590665300+00:00 | gunicorn/run | \n2023-04-09T13:51:27,596955100+00:00 | gunicorn/run | ###############################################\n2023-04-09T13:51:27,602529600+00:00 | gunicorn/run | AzureML Container Runtime Information\nbash: /azureml-env

In [16]:
print(service.scoring_uri)

http://473c49b6-4cfc-4ead-a536-03fa9da6aafa.eastus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [32]:
import json

data = {"data":
        [
          {
            "age": 75,
            "anaemia": 0,
            "creatinine_phosphokinase": 582,
            "diabetes": 0,
            "ejection_fraction": 20,
            "high_blood_pressure": 1,
            "platelets": 265000,
            "serum_creatinine": 1.9,
            "serum_sodium": 130,
            "sex": 1,
            "smoking": 0,
            "time": 4
          },
          {
            "age": 25,
            "anaemia": 1,
            "creatinine_phosphokinase": 425,
            "diabetes": 1,
            "ejection_fraction": 50,
            "high_blood_pressure": 1,
            "platelets": 300000,
            "serum_creatinine": 0.9,
            "serum_sodium": 150,
            "sex": 0,
            "smoking": 1,
            "time": 100
          }
      ]
    }

input_payload = json.dumps(data)

output = service.run(input_payload)

print(output)

[1, 0]


TODO: In the cell below, print the logs of the web service and delete the service

In [18]:
print(service.get_logs())


/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-04-09T13:51:27,572706800+00:00 - rsyslog/run 
2023-04-09T13:51:27,572706800+00:00 - iot-server/run 
2023-04-09T13:51:27,579559700+00:00 - gunicorn/run 
2023-04-09T13:51:27,590665300+00:00 | gunicorn/run | 
2023-04-09T13:51:27,596955100+00:00 | gunicorn/run | ###############################################
2023-04-09T13:51:27,602529600+00:00 | gunicorn/run | AzureML Container Runtime Information
bash: /azureml-envs/azureml_c

In [ ]:
#service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
